In [1]:
import tensorflow as tf

In [2]:
!sudo apt-get install libtinfo5
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtinfo5 is already the newest version (6.2-0ubuntu2).
The following packages were automatically installed and are no longer required:
  accountsservice-ubuntu-schemas bc bluez-obexd cups cups-browsed cups-client
  cups-common cups-core-drivers cups-daemon cups-filters
  cups-filters-core-drivers cups-ipp-utils cups-ppdc cups-server-common
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript
  gir1.2-dbusmenu-glib-0.4 gnome-bluetooth gnome-power-manager
  gnome-screensaver gsettings-ubuntu-schemas gvfs-backends indicator-applet
  indicator-application indicator-appmenu indicator-bluetooth indicator-common
  indicator-datetime indicator-keyboard indicator-messages indicator-power
  indicator-printers indicator-session indicator-sound jayatana
  libaccounts-glib0 libbamf3-2 libcdio-cdda2 libcdio-paranoia2 libcdio18
  libcupsfilters1 libfcitx-config4 libfcitx-gclient1 libfcitx-u

In [3]:
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [5]:
train_data_path ="~/aiffel/dktc/advanced_general_conversation_testB_enlarged.csv"
train_data = pd.read_csv(train_data_path)

In [6]:
train_data['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '일반 대화'],
      dtype=object)

In [7]:
label = {'협박 대화' : 0, '갈취 대화' : 1, '직장 내 괴롭힘 대화' :2, '기타 괴롭힘 대화' : 3, '일반 대화' : 4}

In [8]:
train_data['int_class'] = train_data['class'].map(label)

In [9]:
import re
train_data['conversation'] = train_data['conversation'].apply(lambda x: re.sub(r'[\n…]', '', x))

In [10]:
from transformers import BertTokenizer, TFBertModel

In [60]:
MODEL_NAME = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = TFBertModel.from_pretrained(MODEL_NAME, num_labels=5, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [12]:
# add_tokens = ['[NEW']
new_words = ['구급차', '지혈', '침입', '고속도로', '엘리베이터','진정', '한강공원', '빌라', '소방차','대피', '편의점', '부엌', '심폐소생술', '침착', '출혈', '빌딩', '중학교', '전여친', '전남친', '안부', '지각', '팀장', '##팀장', '팀원', '##팀원', '스캔들', '읽씹', '간식', '프로젝트', '믹스커피', '군것질', '필수품', '힐링', '시발', '씨발', '상사', '피드백', '성의', '레스토랑', '칼퇴', '서운', '퇴사','숙제', '농구', '반장', '쌍욕', '망각', '수다', '떨기', '음료수', '벌점', '지각', '교복', 'PPT', '타이트', '미팅', '희정', '영지', '네일', '파스텔', '자릿세', '보이스피싱', '아이패드', '뺀질', '휠체어', '옥상', '노약자', '##석', '미스##', '데이트', '민아', '말년', '##과장', '과장', '민서', '##씨', '경마장', '숙제', '##대리', '대리', '지수', '교무실', '##주임',
'주임', '진희', '병신', '차장',  '##차장', '사원', '##사원', '이##', '김##', '박##', '가평', '강남', '거창', '관악', '구로', '나주', '대구', '대만', '대전', '독도', '독일', '동해', '로마', '몽골', '미국', '부산', '부여', '북촌', '북한','강민', '관우', '광수', '나은', '민재', '민정', '민지', '민철', '민하', '민호', '민희', '개독', '꼰대', '기레기','구글', '구찌', '농협', '던힐', '레종', '맥북', '샤넬', '힐튼', '나이키', '넥서스', '롤렉스', '리니지', '말보로', '빼빼로', '생로랑', '샤오미']

In [13]:
num_added_toks = tokenizer.add_tokens(new_words)
print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
model.resize_token_embeddings(tokenizer.vocab_size+num_added_toks)

We have added 110 tokens


In [14]:
# 테스트 데이터 토큰화 (문장별 토큰 개수 확인)
tokenized_texts = [tokenizer.tokenize(text) for text in train_data['conversation']]

# 각 문장의 토큰 개수 계산
token_lengths = [len(tokens) for tokens in tokenized_texts]

# 최대 토큰 개수 찾기
max_token_length = max(token_lengths)

print(f"문장에서 가장 긴 토큰 개수: {max_token_length}")

문장에서 가장 긴 토큰 개수: 599


In [15]:
count = 0
for i in range(len(tokenized_texts)):
    if token_lengths[i]>300:
        print(count)
        print(tokenized_texts[i])
        print('\n')
        count += 1

0
['야', '신규', '너', '똑바로', '환자', '관리', '안', '하냐', '?', '인수', '##인', '##계', '제대로', '들은', '##거', '맞아', '?', '너', '누가', '저', '환자', '항', '##암', '약', '##물', '속도', '저렇게', '하', '##래', '죽이고', '싶어서', '환장', '##했어', '?', '인수', '##인', '##계', '들었는데', '.', '어', '.', '약', '##물', '속도', '##는', '제', '프리', '##셉', '##터', '선생님', '##이', '.', '설명', '##해주', '##신', '##바로', '.', '저렇게', '하라고', '하셨', '##어요', '.', '니', '프리', '##셉', '##터', '##가', '문제', '##라는거야', '?', '너가', '잘', '못', '##들은', '##거', '아니고', '?', '아니', '바쁘', '##셔서', '그런지', '.', '책', '보고', '알아서', '하라', '그래서', '책', '보고', '최대한', '아는', '대로', '한', '##겁니다', '.', '모르는', '##거', '물어보', '##라고', '프리', '##셉', '##터', '붙여', '##준거', '아니야', '?', '너', '여기', '실', '##전이', '##야', '니', '하는', '행동에', '환자가', '죽을', '##수도', '있다고', '.', '야', '김', '##간', '##호사', '너', '얘', '프리', '##셉', '##터', '##지', '?', '네', '맞는데', '##요', '왜', '그러', '##시죠', '?', '아니', '얘가', '약', '##물', '조절', '잘못', '##했는데', '너가', '책', '보고', '하라', '그랬다', '##고', '그러', '##던데', '?', '대충', '알려', '##줬', '##냐', '?', '그런',

In [16]:
# 최대 토큰 인텍스, 문장, 토큰 수 확인
max_token_idx = token_lengths.index(max(token_lengths))  # 가장 긴 문장의 인덱스
max_token_text = train_data['conversation'].iloc[max_token_idx]  # 해당 문장
max_token_count = token_lengths[max_token_idx]

In [17]:
max_token_text

' 그래 오늘 날씨가 진짜 좋네. 어때 우리 산책할까? 좋은 생각인 것 같아. 그런데 지금 시간이 어때? 지금 딱 좋아. 한 시간 정도 걸리면 충분할 거야. 좋아 가자. 그런데 우리가 그냥 아무데나 가면 어떡해? 아니야 나는 미리 검색해뒀어. 좋은 산책로가 있대. 그래 그럼 나도 가볼게. 그런데 제발 너무 멀리 가지 말자. 그래 걱정 마. 너무 멀지 않은 곳이니까. 좋아 그럼 지금 바로 갈게. 그런데 내가 오늘 교수님 연구에 참여하느냐 안 하느냐에 따라 시간이 달라질 수 있어. 아니 그럴 필요 없어. 교수님은 네가 오늘 연구에 참여하지 않아도 괜찮다고 하셨어. 진짜? 그럼 다행이야. 그럼 지금 바로 갈게. 좋아 그럼 만나자. 제발 지금 바로 그 문제를 해결해야 해. 너무 늦어지면 안 돼. 그래 하지만 내가 지금 바빠서 바로 처리할 수 없어. 아니야 이건 정말 중요한 문제야. 내가 도와줄까? 아니 괜찮아. 내가 알아서 처리할게. 하지만 진짜 고마워. 그래 그럼 내가 도와줄게. 내가 지금 바로 그 문제를 해결할게. 아니야 정말 고마워. 하지만 내가 해결하는 게 더 좋아. 좋아 그럼 내가 기다릴게. 하지만 너무 늦어지면 안 돼. 그래 알았어. 최대한 빨리 해결하도록 노력할게. 좋아 그럼 지금 바로 다른 일을 처리할게. 그래 고마워. 그 사람 진짜 별로야. 오늘 나한테 너무 무례하게 굴었어. 그래 나도 그 사람한테 좀 실망했어. 아니야 나는 그 사람한테 너무 화가 나. 그래 나도 그 사람한테 실망했어. 하지만 너무 화를 내지는 말자. 아니야 나는 그 사람한테 진짜 사과를 받고 싶어. 그래 나도 그 사람한테 실망했어. 하지만 너무 화를 내지는 말자. 좋아 그럼 내가 그 사람에게 사과를 요구할게. 그래 좋아. 하지만 너무 화를 내지는 말자. 그래 알았어. 내가 그 사람에게 사과를 요구할게. 좋아 그럼 지금 바로 처리하자. 우리 오늘 데이트 어때? 그래 좋은 생각인 것 같아. 하지만 내가 지금 바빠서 어떡해? 아니야 나는 미리 예약해뒀어. 오늘 저녁에 맛있는 음식이 기다리

In [18]:
max_token_count

599

In [27]:
#MAX_LEN = max_token_count + 2 # 문장 최대 토큰 길이
MAX_LEN = 300 # max length of bert model
NUM_LABELS = 5  # 분류할 클래스 개수
BATCH_SIZE = 8
EPOCHS = 2

In [20]:
# 텍스트를 토큰화하는 함수
def tokenize_texts(texts, labels):
    encodings = tokenizer(
        list(texts),  # 리스트 형태로 변환
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN, 
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"], tf.convert_to_tensor(labels, dtype=tf.int32)

In [21]:
# 데이터 분할 (8:2 비율로 train/val 나누기)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['conversation'], train_data['int_class'], test_size=0.2, random_state=42
)

In [22]:
# 데이터 토큰화
train_input_ids, train_attention_mask, train_labels = tokenize_texts(train_texts, train_labels)
val_input_ids, val_attention_mask, val_labels = tokenize_texts(val_texts, val_labels)

In [23]:
# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": train_input_ids, "attention_mask": train_attention_mask}, train_labels)
).shuffle(len(train_texts)).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": val_input_ids, "attention_mask": val_attention_mask}, val_labels)
).batch(BATCH_SIZE)

In [24]:
# 데이터 변환 (train_data['conversation'], train_data['int_class'] 사용)
input_ids, attention_mask, label_tensor = tokenize_texts(train_data['conversation'], train_data['int_class'])

# TensorFlow Dataset 생성
dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": input_ids, "attention_mask": attention_mask}, label_tensor)
)

# 배치 및 섞기
dataset = dataset.shuffle(len(train_data)).batch(BATCH_SIZE)

In [46]:
# EarlyStopping 콜백 설정 (patience=2)
early_stopping = EarlyStopping(
    monitor="val_loss",  # 검증 데이터 손실 기준
    patience=3,  # 3epoch 동안 개선되지 않으면 종료
    restore_best_weights=True  # 가장 성능이 좋았던 가중치 복원
)

In [51]:
# ModelCheckpoint 설정 (최적 모델 저장)
model_checkpoint = ModelCheckpoint(
    "best_kobert_testB_loss_metric.h5",  # 저장할 모델 파일명
    monitor="val_loss",  # 검증 손실 기준으로 저장
    save_best_only=True,  # 가장 성능 좋은 모델만 저장
    save_weights_only=True,  # 가중치만 저장 (전체 모델 저장하려면 False)
    verbose=1  # 저장될 때 로그 출력
)

In [61]:
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = tf.reshape(tf.one_hot(tf.cast(y_true, tf.int32), depth=5), (-1, 5))
        y_pred = tf.reshape(y_pred, (-1, 5))
        ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        p_t = tf.exp(-ce)
        focal_loss = alpha * (1 - p_t) ** gamma * ce
        return tf.reduce_mean(focal_loss)
    return loss

In [62]:
def f1_metric(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=1)
    y_true = tf.reshape(tf.cast(y_true, tf.int32), (-1,))
    return tf.py_function(lambda y_true, y_pred: f1_score(y_true.numpy(), y_pred.numpy(), average='macro'), [y_true, y_pred], tf.double)

In [63]:
input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

# BERT 모델에 입력
bert_output = model(input_ids, attention_mask=attention_mask)[1]
x = Dropout(0.1)(bert_output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(NUM_LABELS, activation="softmax")(x)  # 다중 클래스 분류

# 모델 정의
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=focal_loss(),
    metrics=[f1_metric]
)

# 모델 구조 확인
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 300)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 300)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   TFBaseModelOutputWit 108918528   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dropout_154 (Dropout)           (None, 768)          0           tf_bert_model_3[0][1]      

In [64]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[model_checkpoint]
)

Epoch 1/2
554/554 [==============================] - 329s 566ms/step - loss: 0.0696 - f1_metric: 0.7372 - val_loss: 0.0300 - val_f1_metric: 0.8741

Epoch 00001: val_loss improved from inf to 0.02999, saving model to best_kobert_testB_loss_metric.h5
Epoch 2/2
554/554 [==============================] - 311s 561ms/step - loss: 0.0234 - f1_metric: 0.8975 - val_loss: 0.0337 - val_f1_metric: 0.8656

Epoch 00002: val_loss did not improve from 0.02999


In [65]:
from sklearn.metrics import f1_score

In [66]:
# 검증 데이터 예측
val_predictions = model.predict({"input_ids": val_input_ids, "attention_mask": val_attention_mask})
predicted_classes = np.argmax(val_predictions, axis=1)  # 확률값을 가장 높은 클래스 인덱스로 변환

# F1-score 계산
f1 = f1_score(val_labels, predicted_classes, average="weighted")

print(f"검증 데이터 F1-score: {f1:.4f}")

검증 데이터 F1-score: 0.9088


In [67]:
def tokenize_test_texts(texts):
    encodings = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"]

In [68]:
import os
import pandas as pd
import json

test_json_path = os.getenv('HOME')+'/aiffel/dktc/data/test.json'

# JSON 파일 불러오기
with open(test_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 변환: key를 인덱스로 하고, 내부 "text" 값을 컬럼으로 변환
test_data = pd.DataFrame.from_dict(data, orient="index")
test_data.head()

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그럼 취소할까요 아가씨 내 여기단골이니 담에 갖다줄께 저도 알바생이라 외상안됩니다 아따 누가 떼먹는다고 그러나 갖다준다고 안됩니다 자꾸이럼 경찰불러요 아가씨 담배피는교 그건 왜 물으세요 그람 아가씨 담배 한대만 빌립시다 내 지금 지갑도 잃어버리고 기분이 그래서 그러니 여기요 아따 주는김에 한개더 주면 되겠네
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? 그렇지? 2달만 파견 잘 갔다오면 승진이야. 네? 저는 별로 가고 싶지 않습니다. 여기 있는 모든사람도 가기 싫어해. 그러니까 막내인 영지씨가 가는게 맞지 정말 죄송합니다. 저는 못갑니다. 장난해? 모두를 위해 영지씨가 희생하는게 싫어? 네. 부당한 방법으로 가는 것 같습니다. 영지씨 안가면 회사생활 오래 못할 것 같은데 그래도 안갈거야? 안가면 지옥일텐데. 그래도 이 방법은 아닌 것 같습니다. 죄송합니다.
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 행실이 맘에 안들어 네 실천할께요 옆에 동기들 좀 본받으란 말야 어? 네 알겠습니다 그래 똑바로해 노럭할께요
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도 말해야지? .켰습니다. 주인님. 오냐. 앞으로도 잘부탁한다 와이파이. 넵. 이렇게만 해? 알겠지? 응.


In [69]:
test_input_ids, test_attention_mask = tokenize_test_texts(test_data['text'])

In [70]:
# 모델 예측 수행
predictions = model.predict({"input_ids": test_input_ids, "attention_mask": test_attention_mask})

# 확률값을 가장 높은 클래스 인덱스로 변환
predicted_classes = np.argmax(predictions, axis=1)

# 결과를 데이터프레임에 추가
test_data['target'] = predicted_classes

In [71]:
# 예측된 결과 출력
print(test_data[['text', 'target']])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        text  \
t_000                                                                                                                                                                                                                                                                                                                                                                                         아가씨 담배한갑주소

In [72]:
test_data['target'].value_counts()

3    164
2    123
1     95
0     88
4     30
Name: target, dtype: int64

In [73]:
# 긴 텍스트 출력 제한 해제
pd.options.display.max_colwidth = None  
# 예측된 결과 출력
display(test_data[test_data['target'] == 4])

,text,target
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래,4
t_013,과장님 오늘 부장님 기분이 안 좋으신것 같네요 오늘 프레젠테이션 있었는데 사장님한테 엄청 까였나봐 아 그래요 오늘 조심해야겠네요 엉 알아서들 조심해 완전 저기압인거 같아 네 근데 오늘 저희 주간 회의 있어서 어차피 부딪힐거 같은데 그때까지 부장님 기분이 풀리길 바래야지 뭐 아 오늘 잘못하면 엄청 혼날꺼 같은데 그니까 준비 잘해 네 알겠습니다,4
t_057,아니 진짜 걍 지구 망할 거 같아 위쪽은 비 난리고 밑에는 비는 안 와도 공기 난리고 이제 또 비오는 너무 많이와 진짜 망할려고이러나 그러게 말이여 진짜 몇년 전부터 미세먼지도 난리고 원래는 봄에만 황사처럼 공기 안 좋았잖아 근데 요즘엔 걍 사계절 내내 안 좋으니까 약간 더 무서운 건 또 거기에 다 적응한 거임 일단 나부터 적응 다함 공기 안 좋은 거 그러려니해 아 개인정 어쩌다 이렇게 됐는지 진짜 예전이 그립다 초딩 때라던가 그냥 벌받는거 같기두 하구 마잡 그래서 나 늦은 감 있지만 이제라도 일회용 줄이려고 뭔가 이젠 다들 좀 경각심 느껴야 할 거 같음\n,4
t_068,"옥상에서 사람이 떨어졌어요. 사람이 떨어졌다고요? 위치가 어디세요? 경기도 광주시 오포읍 문형리 행복 아파트인데요, 사람이 떨어졌어요. 몇층에서 떨어졌는지는 보셨나요? 아니요 어디서 떨어졌는지는 못봤어요. 알겠습니다. 잠시만 기다려 주세요.",4
t_082,"천안 두정동 제이빌라 인데요, 지금 화재가 발생했어요. 빌라 어디에 불이 났나요? 1층 주차장에서 불이 나고 있어요. 그러면 지금 다친 사람도 있나요? 아직까지는 없는거 같아요. 알겠습니다. 지금 119 소방대를 보내도록 하겠습니다.",4
t_104,너 꿈이 뭐라고? 교수? 넵 네 머리로 무슨 교수는 교수야. 제 목표입니다. 현실성 없는 목표는 그냥 없는 것보다 못해. 포기해. 저는 제 꿈을 계속 실현해 나갈 것입니다. 그래 응원은 못해주겠다. 열심히 하겠습니다. 백날 해봐라 교수의 교 자도 못 따라갈거다. 그냥 저를 좀 응원해주셨으면 좋겠네요.,4
t_108,야 너 살 좀 빠졌냐 아니 그대론데 아 그래 좀 빠진 것 같네 요즘 운동하기는 하는데 잘 안빠져 일주일에 몇번 가 나 두번정도 별로 안가네 더 많이 가야지 그런가 겨울이면 원래 더 안빠지는거 아냐 에이 그런가 모르겠네 더 열심히 해봐 알았어 고맙다,4
t_139,지금 사장딸 없어서 에어팟끼고 동영상 보고 있어 사장 심부름 간다고 갔는데 아직도 걔 땡땡이치네 바로 집갔을수도 있고 하 상상하니까 열받네 아냐 오늘 금요일이잖아 우리 즐겁게 지내자 알겠엉\n,4
t_152,"시신을 발견했습니다. 위치가 어디시죠? 인천 영흥도인근 해상인데요, 검은색 옷을 입은 남자가 떠있는것을 발견했어요. 그럼 가까운 지상으로 시신을 가져와주실수 있나요? 네 영흥도 부둣가로 갈게요 알겠습니다.",4
t_162,회사에서 성희롱 타깃이 된 것 같아서 우울해. 직장 내 성희롱 때문에 우울하시군요. 좀 더 이야기 해 주시겠어요? 다른 직원들이 옆자리 직원을 성희롱 하는 게 보여서 도와줬더니 이번엔 내가 타깃이 되었어. 어떻게 하면 성희롱 문제를 해결할 수 있을까요? 내일 직장 내 성희롱 관련 담당자를 찾아가봐야겠어. 일이 잘 풀려서 우울한 기분이 나아지면 좋겠어요.,4


In [75]:
output_df = test_data[['target']].reset_index()
output_df.rename(columns={'index': 'idx'}, inplace=True)

output_df.head()

,idx,target
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,4
4,t_004,3


In [76]:
csv_path = os.getenv('HOME')+'/aiffel/dktc/submission_kobert_testB_loss_metric.csv'
output_df.to_csv(csv_path, index=False)

In [77]:
test_data.to_csv(os.getenv('HOME')+'/aiffel/dktc/result_kobert_testB_loss_metric.csv', index=False)